In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('asd_data.xls')

In [3]:
tabla_susceptibilidades = df.iloc[18:31, :-2]

In [4]:
headers = ['ANTIBIOTICO']
cantidad_de_cepas = int((len(tabla_susceptibilidades.columns) - 1)/ 2)
for i in range(cantidad_de_cepas):
    headers.append(f'Cepa {i + 1}')
    headers.append(f'CIM {i + 1}')

tabla_susceptibilidades.columns = headers

In [5]:
tabla_susceptibilidades = tabla_susceptibilidades.set_index('ANTIBIOTICO')

In [6]:
tablas_cepas = []
for i in range(cantidad_de_cepas):
    indice_inicial = i * 2
    indice_final = indice_inicial + 2
    df = tabla_susceptibilidades.iloc[:, indice_inicial: indice_final]
    df.set_index(tabla_susceptibilidades.index)
    tablas_cepas.append(df)

tablas_cepas


[                                Cepa 1      CIM 1
 ANTIBIOTICO                                      
 Clindamicina                  Sensible      <=0,5
 Ciprofloxacino                Sensible      <=0,5
 Eritromicina                Resistente         >4
 Gentamicina                   Sensible        <=2
 Oxacilina                   Resistente         >2
 Rifampicina                   Sensible      <=0,5
 Sulfametoxazol/Trimetoprim    Sensible  <=0,5/9,5
 Vancomicina                   Sensible          1
 Amikacina                          NaN        NaN
 Ceftazidima                        NaN        NaN
 Cefepime                           NaN        NaN
 Imipenem                           NaN        NaN
 Piperacilina/Tazobactam            NaN        NaN,
                                 Cepa 2      CIM 2
 ANTIBIOTICO                                      
 Clindamicina                       NaN        NaN
 Ciprofloxacino              Resistente         >2
 Eritromicina                 

In [7]:
tablas_cepas[1].transpose()

ANTIBIOTICO,Clindamicina,Ciprofloxacino,Eritromicina,Gentamicina,Oxacilina,Rifampicina,Sulfametoxazol/Trimetoprim,Vancomicina,Amikacina,Ceftazidima,Cefepime,Imipenem,Piperacilina/Tazobactam
Cepa 2,NaN,Resistente,NaN,Resistente,NaN,NaN,Sensible,NaN,Resistente,Sensible,Resistente,Sensible,Sensible
CIM 2,NaN,>2,NaN,>8,NaN,NaN,"<=0,5/9,5",NaN,>32,8,>16,<=2,<=4/4
